In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import torch    
import cv2
import numpy as np
import matplotlib.pyplot as plt
import requests
import torchvision.transforms as transforms
from pytorch_grad_cam import EigenCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image
from PIL import Image
from ultralytics import YOLO


In [6]:
COLORS = np.random.uniform(0, 255, size=(80, 3))

def parse_detections(results):
    detections = results.pandas().xyxy[0]
    detections = detections.to_dict()
    boxes, colors, names = [], [], []

    for i in range(len(detections["xmin"])):
        confidence = detections["confidence"][i]
        if confidence < 0.2:
            continue
        xmin = int(detections["xmin"][i])
        ymin = int(detections["ymin"][i])
        xmax = int(detections["xmax"][i])
        ymax = int(detections["ymax"][i])
        name = detections["name"][i]
        category = int(detections["class"][i])
        color = COLORS[category]

        boxes.append((xmin, ymin, xmax, ymax))
        colors.append(color)
        names.append(name)
    return boxes, colors, names


def draw_detections(boxes, colors, names, img):
    for box, color, name in zip(boxes, colors, names):
        xmin, ymin, xmax, ymax = box
        cv2.rectangle(
            img,
            (xmin, ymin),
            (xmax, ymax),
            color, 
            2)

        cv2.putText(img, name, (xmin, ymin - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2,
                    lineType=cv2.LINE_AA)
    return img


In [ ]:
model = YOLO('/Users/huynhanhkiet/Desktop/Object_Detection/ArTaxOr/yolov10/runs/detect/train/weights/best.pt')

In [27]:
target_layers = [model.model.model[-4]]

In [34]:
target_layers

[SCDown(
   (cv1): Conv(
     (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
     (act): SiLU(inplace=True)
   )
   (cv2): Conv(
     (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128, bias=False)
     (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
     (act): Identity()
   )
 )]

In [33]:
principal_comp = [7,8,9]
cam = EigenCAM(model, target_layers)
grayscale_cam = cam(
    img,
    eigen_smooth=True,
    principal_comp=principal_comp,
)

for i in range(grayscale_cam.shape[3]):
    cam_image = show_cam_on_image(img, grayscale_cam[0,:,:,i], use_rgb=True)
    print(f"principal component {principal_comp[i]+1}")
    plt.imshow(cam_image)
    plt.show()

New https://pypi.org/project/ultralytics/8.3.33 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.31 🚀 Python-3.10.15 torch-2.4.1 CPU (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=/Users/huynhanhkiet/Desktop/Object_Detection/ArTaxOr/yolov10/runs/detect/train/weights/best.pt, data=/kaggle/working/data/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=Fa

RuntimeError: Dataset '/kaggle/working/data/data.yaml' error ❌ '/kaggle/working/data/data.yaml' does not exist